Emil Enikeev, 2022

Paper: https://scholarship.claremont.edu/cgi/viewcontent.cgi?article=3517&context=cmc_theses

# imports

In [ ]:
!pip install empyrical

In [ ]:
import warnings
import itertools
from copy import deepcopy

import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.graph_objects import Layout
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from empyrical import roll_sharpe_ratio, sharpe_ratio, max_drawdown, cum_returns_final

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
pd.options.mode.chained_assignment = None

warnings.filterwarnings("ignore")

In [ ]:
!gdown 1t7plTf_du6XMq0YL_hMpul51AuULY3QG

Downloading...
From: https://drive.google.com/uc?id=1t7plTf_du6XMq0YL_hMpul51AuULY3QG
To: /content/stock_data.csv
100% 1.54G/1.54G [00:13<00:00, 118MB/s] 


# plot tools

In [ ]:
def plotly_lines(df, title=None, legend_title=None):
    if isinstance(df, pd.Series):
        df = df.to_frame(df.name)
    fig = go.Figure(layout=Layout(title=title,
                                  yaxis={'zeroline': False}, ))
    fig.update_xaxes(showgrid=True, gridwidth=1)
    fig.update_yaxes(showgrid=True, gridwidth=1)
    columns = list(df.columns)
    for c in range(len(columns)):
        fig.add_trace(
            go.Scatter(
                x=df[columns[c]].index,
                y=df[columns[c]],
                name=columns[c],
                mode='lines',
                line=dict(
                    width=2)
            )
        )
    fig.update_layout(
        title_font=dict(
            size=26,
        ),
        legend_font=dict(
            size=12,
        ),
        font=dict(
            family="Courier New, monospace",
            size=13,
            color="black"
        ),
        legend_title=legend_title,
    )
    fig.update_layout(
        autosize=False,
        width=640,
        height=360, )
    fig.update_layout(
        margin=dict(
            l=20,
            r=50,
            b=20,
            t=70,
        ),
    )
    return fig

# train valid test split

In [ ]:
stock_data = pd.read_csv('stock_data.csv', parse_dates=['date']).set_index('date')

Изначально были выданы временные промежутки для проведения эксперимента, но, для актуальности, только 2020 год будет использован для кластеризации, а 2021 для построения портфеля и подбора гиперпараметров. Далее лучшие значения будут использованы для бэктеста на 2022 году.

In [ ]:
train_data = stock_data[(stock_data.index >= '2020-01-01') & (stock_data.index <= '2020-12-31')]
# putted more data to valid, to include roll sharpe window period
# after calculating it, df would include dates >= '2021-01-01' only
valid_data = stock_data[(stock_data.index >= '2020-01-01') & (stock_data.index <= '2021-12-31')]
test_data = stock_data[stock_data.index >= '2021-01-01']

In [ ]:
returns = train_data.groupby(['date', 'ticker']).aggregate({'close': 'last'}).stack().unstack(
    level=1).reset_index().set_index('date').drop('level_1', axis=1).ffill()
returns = returns.pct_change().iloc[1:]

In [ ]:
scaled_returns = pd.DataFrame((standard_scaler.fit_transform(returns)), columns=returns.columns).T
scaled_returns.shape

(4115, 242)

In [ ]:
scaled_returns = scaled_returns.dropna()
scaled_returns.shape

(3709, 242)

# find best number of clusters

Finding how many clusters we need using mean silhouette scores

In [ ]:
silhouette_scores = []
number_of_experiments = 100
for n_clusters in tqdm(range(4, 11 + 1)):
    for _ in range(number_of_experiments):
        clusterer = KMeans(algorithm="elkan", n_clusters=n_clusters)
        preds = clusterer.fit_predict(scaled_returns)
        score = silhouette_score(scaled_returns, preds)
        silhouette_scores.append({'n_clusters': n_clusters, 'silhouette_score': score})
scores = pd.DataFrame(silhouette_scores).set_index('n_clusters')

100%|██████████| 8/8 [32:49<00:00, 246.23s/it]


In [ ]:
plotly_lines(scores.groupby('n_clusters').mean(), title='Silhouette scores for clusters')

6 clusters has the best silhouette score, so we choose this number of clusters

# tools for clustering, portfolio build and performance measure

In [ ]:
def get_scaled(data):
    scaled = pd.DataFrame((standard_scaler.fit_transform(data)), columns=data.columns).T
    return scaled.dropna()

In [ ]:
def stock_clustering(scaled_returns, clusterer):
    clusterer.fit(scaled_returns)
    y_clusterered = clusterer.predict(scaled_returns)
    # clustered_tickers: DataFrame, index: ticker, column 'cluster_label': cluster
    clustered_tickers = pd.DataFrame(y_clusterered, columns=['cluster_label'], index=scaled_returns.index)
    # cluster_ticker_dict: Dict, key: cluster, value: list of ticker
    cluster_ticker_dict = clustered_tickers.reset_index().groupby('cluster_label').ticker.apply(list).to_dict()
    return cluster_ticker_dict

In [ ]:
def build_portfolio(returns_df, roll_factor_func, factor_returns, window_roll_factor, window_rebalance, init_date,
                    cluster_ticker_dict):
    all_clusters_len = len(list(itertools.chain.from_iterable(cluster_ticker_dict.values())))

    def portfolio_rebalances(row):
        # stock_weights: dict, key: ticker, value: weight
        stock_weights = dict()
        all_sharpe = row.sort_values(ascending=False)
        for cluster in cluster_ticker_dict.values():
            # we pick one stock from each cluster with highest Sharpe ratio and give it weight corresponding to the cluster size
            top_tickers = all_sharpe.loc[all_sharpe.index.intersection(cluster)].dropna().sort_values(
                ascending=False).index
            if len(top_tickers) > 0:
                stock_weights[top_tickers[0]] = len(
                    cluster) / all_clusters_len
        # to be alligned into dataframe row correctly, dict -> Series
        return pd.Series(stock_weights)

    if factor_returns is None:
        factor_returns = returns_df.copy()
    for ticker in returns_df.columns:
        if roll_factor_func:
            if ticker in factor_returns.columns:
                factor_returns[ticker] = roll_factor_func(factor_returns[ticker].dropna(), window_roll_factor)
        else:
            factor_returns[ticker] = factor_returns[ticker].dropna()

    factor_returns = factor_returns[factor_returns.index > init_date]
    returns_df = returns_df[returns_df.index > init_date]
    rebalance_dates = returns_df.index[::window_rebalance]

    # in rebalance_weights only stocks that occur in portfolio at least once
    rebalance_weights = factor_returns.loc[rebalance_dates].apply(lambda row: portfolio_rebalances(row), axis=1).fillna(
        0)

    # reindex with all dates, not only rebalance dates, and ffill
    rebalance_weights_full = rebalance_weights.reindex(
        returns_df.index[returns_df.index >= rebalance_weights.index[0]]).ffill()

    # non zero weights -> 1
    rebalance_weights_full_mask = rebalance_weights_full.mask(rebalance_weights_full > 0, 1)

    returns_df = returns_df[returns_df.index >= rebalance_weights.index[0]]

    # remove returns of tickers that never occur in portfolio
    # * mask -> stocks that not in portfolio got 0 return
    portfolio_ticker_returns = returns_df[rebalance_weights_full_mask.columns] * rebalance_weights_full_mask

    # portfolio returns, but if we did rebalance every <index date>
    nonreb_pf_returns = (1 + (rebalance_weights_full * portfolio_ticker_returns).sum(axis=1))

    # weights change between rebalances
    weights_multiplicator = (1 + portfolio_ticker_returns.mask(portfolio_ticker_returns == 0, -1)).div(
        nonreb_pf_returns,
        axis=0)

    # calculate correct portfolio returns considering rebalances
    portfolio_daily_returns = (
            1 + (weights_multiplicator * rebalance_weights_full * portfolio_ticker_returns).sum(axis=1)).cumprod()
    portfolio_daily_returns = portfolio_daily_returns / portfolio_daily_returns.iloc[0]
    return portfolio_daily_returns, rebalance_weights

In [ ]:
def cumulative_to_non(cumulative_returns: pd.Series) -> pd.Series:
    non_cumulative_returns = cumulative_returns / cumulative_returns.shift(1) - 1
    return non_cumulative_returns.dropna()

In [ ]:
def portfolio_performance(portfolio_returns: pd.DataFrame, is_cumulative=True) -> pd.DataFrame:
    if isinstance(portfolio_returns, pd.Series):
        portfolio_returns = portfolio_returns.to_frame(portfolio_returns.name)
    performance_report = list()
    for strategy in portfolio_returns.columns:
        if is_cumulative:
            returns = cumulative_to_non(portfolio_returns[strategy])
        else:
            returns = portfolio_returns[strategy]
        performance_metrics = dict()
        performance_metrics['sharpe_ratio'] = sharpe_ratio(returns)
        performance_metrics['max_drawdown'] = max_drawdown(returns)
        performance_metrics['return_to_drawdown'] = cum_returns_final(returns) / abs(
            performance_metrics['max_drawdown'])
        performance_report.append(performance_metrics)
    return pd.DataFrame(performance_report, index=portfolio_returns.columns).sort_values(['return_to_drawdown'],
                                                                                         ascending=False)

# testing clustering on close return data

In [ ]:
scaled_returns = get_scaled(returns)

In [ ]:
cluster_ticker_dict = stock_clustering(scaled_returns, clusterer=KMeans(6))

In [ ]:
returns_valid = valid_data.groupby(['date', 'ticker']).aggregate({'close': 'last'}).stack().unstack(
    level=1).reset_index().set_index('date').drop('level_1', axis=1).ffill()
# returns_valid: DataFrame, index: date, columns: <tickers>, cells: unscaled returns
returns_valid = returns_valid.ffill().pct_change().iloc[1:]

In [ ]:
portfolio_daily_returns, rebalance_weights = build_portfolio(returns_df=returns_valid,
                                                             roll_factor_func=roll_sharpe_ratio,
                                                             factor_returns=None,
                                                             window_roll_factor=20,
                                                             window_rebalance=20,
                                                             init_date='2021-01-01',
                                                             cluster_ticker_dict=cluster_ticker_dict)

In [ ]:
plotly_lines(portfolio_daily_returns, 'Portfolio after clustering on returns')

In [ ]:
portfolio_performance(portfolio_daily_returns)

,sharpe_ratio,max_drawdown,return_to_drawdown
0,2.676157,-0.162476,10.964458


In [ ]:
# portfolio_daily_returns.to_excel('portfolio_returns.xlsx')

# testing clustering on financial ratios

In [ ]:
cluster_dicts = dict()
daily_factors = ['close', 'pe', 'pe_ttm', 'pb', 'ps', 'dv_ratio', 'dv_ttm']
# only factors that changes every day
for f in tqdm(daily_factors):
    t = valid_data.groupby(['date', 'ticker']).aggregate({f: 'last'}).stack().unstack(
        level=1).reset_index().set_index('date').drop('level_1', axis=1).ffill()
    t = t.ffill().pct_change().iloc[1:]
    try:
        scaled = get_scaled(t)
    except ValueError:
        print(f"Factor [ {f} ] contains contains infinity or a value too large for dtype('float64')")
    cluster_ticker_dict_t = stock_clustering(scaled, clusterer=KMeans(6))
    cluster_dicts[f] = cluster_ticker_dict_t

 71%|███████▏  | 5/7 [00:26<00:11,  5.69s/it]

Factor [ dv_ratio ] contains contains infinity or a value too large for dtype('float64')


100%|██████████| 7/7 [00:37<00:00,  5.29s/it]


In [ ]:
constant_params = {'returns_df': returns_valid,
                   'roll_factor_func': roll_sharpe_ratio,
                   'factor_returns': None,
                   'window_rebalance': 20,
                   'init_date': '2021-01-01'}

In [ ]:
test_grid = list()
strategy_names = list()
for window_roll_factor in list(range(20, 120, 20)):
    for factor, f_returns in cluster_dicts.items():
        params = deepcopy(constant_params)
        params['window_roll_factor'] = window_roll_factor
        params['cluster_ticker_dict'] = f_returns
        test_grid.append(params)
        strategy_names.append(f'{factor}_{window_roll_factor}')

In [ ]:
compare_data = list()
for params in tqdm(test_grid):
    portfolio_daily_returns, _ = build_portfolio(**params)
    compare_data.append(portfolio_daily_returns.to_dict())

100%|██████████| 35/35 [04:23<00:00,  7.52s/it]


In [ ]:
compare_df = pd.DataFrame(compare_data).T
compare_df.columns = strategy_names

In [ ]:
plotly_lines(compare_df, title='Compare Financial Ratios & windows')

In [ ]:
# compare_df.to_excel('compare_ratios_windows.xlsx')

In [ ]:
portfolio_performance(compare_df).head()

,sharpe_ratio,max_drawdown,return_to_drawdown
pb_40,3.367518,-0.139089,16.397254
close_60,2.865318,-0.173549,9.410250
pb_20,2.817356,-0.233663,8.441958
close_100,3.060657,-0.193863,7.681373
ps_20,2.561714,-0.232377,7.113113


# backtest on 2022 using best parameters

In [ ]:
returns_test = test_data.groupby(['date', 'ticker']).aggregate({'close': 'last'}).stack().unstack(
    level=1).reset_index().set_index('date').drop('level_1', axis=1).ffill()
returns_test = returns_test.ffill().pct_change().iloc[1:]

In [ ]:
pb_valid = test_data.groupby(['date', 'ticker']).aggregate({'pb': 'last'}).stack().unstack(
    level=1).reset_index().set_index('date').drop('level_1', axis=1).ffill()
pb_valid = pb_valid.ffill().pct_change().iloc[1:]
pb_valid = pb_valid[pb_valid.index > '2021-01-01']

In [ ]:
scaled_returns_valid = get_scaled(pb_valid)

In [ ]:
cluster_ticker_test = stock_clustering(scaled_returns_valid, clusterer=KMeans(6))

In [ ]:
portfolio_daily_returns, rebalance_weights = build_portfolio(returns_df=returns_test,
                                                             roll_factor_func=roll_sharpe_ratio,
                                                             factor_returns=None,
                                                             window_roll_factor=40,
                                                             window_rebalance=20,
                                                             init_date='2022-01-01',
                                                             cluster_ticker_dict=cluster_ticker_test)

In [ ]:
plotly_lines(portfolio_daily_returns, title='Test')

In [ ]:
portfolio_performance(portfolio_daily_returns)

,sharpe_ratio,max_drawdown,return_to_drawdown
0,-1.186349,-0.421731,-0.432868
